Estandarizacion

In [1]:
import math
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.layers import Dropout
from keras.layers import BatchNormalization
import requests
from bs4 import BeautifulSoup
import os
from astropy.io import fits
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table
from tqdm import tqdm
import time
from glob import glob
from astropy.utils.data import get_pkg_data_filename
from astropy.convolution import convolve, Gaussian1DKernel

Calculamos la lista de listas con los promedios y desviaciones estandar para cada uno de los pixeles en B,Z,R para los 11 fits

In [2]:
B_mean=[]# en estas listas se guardaran todos los espectros
Z_mean=[]
R_mean=[]
B_var = []  # Lista para guardar las varianzas
Z_var = []
R_var = []
N_espectros= 0 #Este se utiliza para calcular la media de cada pixel
N_espectros_menos_1= 0 #Este se utiliza para calcular la varianza agrupada 

n=0

archivos= ['DataDESI_1_36.fits', 'DataDESI_37_72.fits', 'DataDESI_73_108.fits', 'DataDESI_109_144.fits', 'DataDESI_145_180.fits', 'DataDESI_181_216.fits'
 , 'DataDESI_217_252.fits', 'DataDESI_253_288.fits', 'DataDESI_289_324.fits', 'DataDESI_325_360.fits', 'DataDESI_361_379.fits']

for h in range(len(archivos)):
    espc = fits.open(archivos[h]) #open file
    len_espc= len(espc[2].data)
    
    #leemos la informacion
    Bflux=espc[2].data
    Rflux=espc[3].data
    Zflux=espc[4].data

    promedios_b = np.mean(Bflux, axis=0)
    promedios_r = np.mean(Rflux, axis=0)
    promedios_z = np.mean(Zflux, axis=0)

    promedios_b_promedio = [arr * (len_espc) for arr in promedios_b] #Estas guardan NhPh
    promedios_r_promedio = [arr * (len_espc) for arr in promedios_r]
    promedios_z_promedio = [arr * (len_espc) for arr in promedios_z]

    N_espectros+= len_espc #Va sumando los espectros para tener en cuenta la muestra total

    B_mean.append(promedios_b_promedio) #Estas guardan N1P1,N2P2,...,N11P11
    Z_mean.append(promedios_z_promedio)
    R_mean.append(promedios_r_promedio)


    varianza_b = np.var(Bflux, axis=0)
    varianza_r = np.var(Rflux, axis=0)
    varianza_z = np.var(Zflux, axis=0)

    varianza_b_varianza = [arr * (len_espc-1) for arr in varianza_b] #Estas guardan (Nh-1)sigmah
    varianza_r_varianza = [arr * (len_espc-1) for arr in varianza_r]
    varianza_z_varianza = [arr * (len_espc-1) for arr in varianza_z]

    N_espectros_menos_1+= len_espc-1

    B_var.append(varianza_b_varianza) #Estas guardan (N1-1)sigma1,(N2-1)sigma2,...,(N11-1)sigma11
    Z_var.append(varianza_z_varianza)
    R_var.append(varianza_r_varianza)
    
    n+=1
    print(n)

1
2
3
4
5
6
7
8
9
10
11


Hacemos el promedio de promedios

In [3]:
ponderacion_B = [sum(valores) for valores in zip(*B_mean)] #Suma N1P1+N2P2+...+N11P11 solo falta dividir entre la cantidad total
ponderacion_Z = [sum(valores) for valores in zip(*Z_mean)]
ponderacion_R = [sum(valores) for valores in zip(*R_mean)]

media_pixel_B = np.array([elemento / N_espectros for elemento in ponderacion_B])#Estas son las listas con los valores promedios
media_pixel_Z = np.array([elemento / N_espectros for elemento in ponderacion_Z])# en cada pixel
media_pixel_R = np.array([elemento / N_espectros for elemento in ponderacion_R])

In [4]:
print(len(media_pixel_B))
print(len(media_pixel_Z))
print(len(media_pixel_R))

2751
2881
2326


Hacemos la varianza agrupada

In [5]:
var_agrupada_B = [sum(valores) for valores in zip(*B_var)] #Suma (N1-1)sigma1+(N2-1)sigma2+...+(N11-1)sigma11
var_agrupada_Z = [sum(valores) for valores in zip(*Z_var)]
var_agrupada_R = [sum(valores) for valores in zip(*R_var)]

var_B = np.array([elemento / N_espectros_menos_1 for elemento in var_agrupada_B])#Estas son las listas con las varianzas
var_Z = np.array([elemento / N_espectros_menos_1 for elemento in var_agrupada_Z])# en cada pixel
var_R = np.array([elemento / N_espectros_menos_1 for elemento in var_agrupada_R])

Con las varianzas agrupadas calculamos las desviaciones

In [6]:
desv_B = np.sqrt(var_B) #Estas son las listas con las desviaciones en cada pixel
desv_Z = np.sqrt(var_Z)
desv_R = np.sqrt(var_R)

In [7]:
print(len(desv_B))
print(len(desv_Z))
print(len(desv_R))

2751
2881
2326


Ahora, editamos los espectros y creamos nuevos archivos .fits con los espectros estandarizados

In [8]:
#archivos= ['DataDESI_1_36.fits', 'DataDESI_37_72.fits', 'DataDESI_73_108.fits', 'DataDESI_109_144.fits', 'DataDESI_145_180.fits', 'DataDESI_181_216.fits'
# , 'DataDESI_217_252.fits', 'DataDESI_253_288.fits', 'DataDESI_289_324.fits', 'DataDESI_325_360.fits', 'DataDESI_361_379.fits']

archivos= ['DataDESI_361_379.fits']

for h in range(len(archivos)):
    espc = fits.open(archivos[h])
    
    B_FLUX_STAN=[]
    Z_FLUX_STAN=[]
    R_FLUX_STAN=[]
    
    for i in range(len(espc[4].data)):
        espc_b=((espc[2].data[i])-media_pixel_B)/desv_B
        espc_z=((espc[4].data[i])-media_pixel_Z)/desv_Z
        espc_r=((espc[3].data[i])-media_pixel_R)/desv_R
    
        B_FLUX_STAN.append(espc_b)
        Z_FLUX_STAN.append(espc_z)
        R_FLUX_STAN.append(espc_r)
        print("copiado_1")
    
    B_FLUX_STAN=np.array(B_FLUX_STAN, dtype=np.float32)
    Z_FLUX_STAN=np.array(Z_FLUX_STAN, dtype=np.float32)
    R_FLUX_STAN=np.array(R_FLUX_STAN, dtype=np.float32)
    
    
    ##### AQUI DEBEMOS CREAR UN ARCHIVO.FITS EN DONDE VUELVA A GUARDAR LA MISMA TABLA DE LA OTRA VEZ Y AHORA B_FLUX_STAN, Z_FLUX_STAN, R_FLUX_STAN.
    # Crea un objeto ImageHDU
    hdub = fits.ImageHDU(B_FLUX_STAN)
    hduz = fits.ImageHDU(Z_FLUX_STAN)
    hdur = fits.ImageHDU(R_FLUX_STAN)
    
    # Establece el nombre del HDU como 'B_FLUX'
    hdub.name = 'B_FLUX_S'
    hdur.name = 'R_FLUX_S'
    hduz.name = 'Z_FLUX_S'
    
    #Agrega al fits.
    nombre_archivo = 'DataDESI.fits'
    hdulist = fits.open(nombre_archivo, mode='append')
    bintable_hdu = fits.BinTableHDU(espc[1].data)
    bintable_hdu.header = espc[1].header
    hdulist.append(bintable_hdu)
    hdulist.append(hdub)
    hdulist.append(hdur)
    hdulist.append(hduz)
    print("Copiado")
    hdulist.close() 

copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1
copiado_1


In [9]:
from astropy.table import Table
espc = fits.open('DataDESI.fits')
espc.info()
print(len(espc[2].data))

Filename: DataDESI.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     14   21772R x 3C   [7A, D, D]   
  2  B_FLUX_S      1 ImageHDU         8   (2751, 21772)   float32   
  3  R_FLUX_S      1 ImageHDU         8   (2326, 21772)   float32   
  4  Z_FLUX_S      1 ImageHDU         8   (2881, 21772)   float32   
21772


In [10]:
Table.read(espc, hdu=1) 

SPECTYPE,Z,TARGETID
str7,float64,float64
GALAXY,0.8096589134393345,3.963312758430371e+16
GALAXY,0.9282989193404377,3.963313202187755e+16
GALAXY,1.5251344667187916,3.963313643848008e+16
GALAXY,0.25679214910795595,3.963313202187697e+16
GALAXY,0.7563224362494237,3.963312758430276e+16
STAR,-0.0011996796199236536,3.9633136442672536e+16
QSO,1.1807410029513217,3.963313202187771e+16
QSO,0.9350379047671412,3.9633127584302856e+16
GALAXY,0.8423552934538804,3.9633127584303496e+16


In [11]:
espc[2].data

array([[-0.2592579 , -0.4471716 , -1.2270695 , ..., -0.29687297,
        -0.26531804, -0.06525054],
       [-0.47962636,  0.7025317 ,  0.297829  , ..., -0.32085192,
        -0.06278338, -0.31165105],
       [-0.47678852, -0.1241371 ,  0.19980276, ..., -0.04245101,
         0.12616923,  0.00268243],
       ...,
       [ 0.39020112, -0.67205286, -0.48976108, ...,  0.22609124,
         0.24696113,  0.5756739 ],
       [-0.12779088, -0.12648624, -0.12772645, ..., -0.15195711,
        -0.15201768, -0.15183865],
       [-0.1582803 , -0.01560504,  0.71921134, ..., -0.12556091,
        -0.12939003, -0.07962684]], dtype=float32)

In [12]:
from astropy.table import Table
espc = fits.open('DataDESI_361_379.fits')
espc.info()
print(len(espc[2].data))

Filename: DataDESI_361_379.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     14   21772R x 3C   [7A, D, D]   
  2  B_FLUX        1 ImageHDU         8   (2751, 21772)   float32   
  3  R_FLUX        1 ImageHDU         8   (2326, 21772)   float32   
  4  Z_FLUX        1 ImageHDU         8   (2881, 21772)   float32   
21772


In [13]:
espc[2].data

array([[-0.8759859 , -2.1978185 , -7.528637  , ..., -0.71444756,
        -0.56100196,  0.433785  ],
       [-2.3443363 ,  5.6816783 ,  2.9143338 , ..., -0.83266574,
         0.44183993, -0.80062056],
       [-2.3254273 ,  0.01609989,  2.24302   , ...,  0.5398743 ,
         1.3774309 ,  0.77411234],
       ...,
       [ 3.4514637 , -3.7390435 , -2.4793239 , ...,  1.8638103 ,
         1.9755269 ,  3.6446576 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.2031559 ,  0.7599248 ,  5.800089  , ...,  0.13013548,
         0.11203983,  0.36176327]], dtype=float32)